In [1]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.layers import Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import optimizers
import tensorflow as tf


import keras.backend as K

Using TensorFlow backend.


In [2]:
from utils import *
df = functions.parse_file(r"raw_data/EmoContext/train.txt", "EmoContext")
df.head(20)

,id,turn1,turn2,turn3,label
0,0,Don't worry I'm girl,hmm how do I know if you are,What's ur name?,others
1,1,When did I?,saw many times i think -_-,No. I never saw you,angry
2,2,By,by Google Chrome,Where you live,others
3,3,U r ridiculous,I might be ridiculous but I am telling the truth.,U little disgusting whore,angry
4,4,Just for time pass,wt do u do 4 a living then,Maybe,others
5,5,I'm a dog person,youre so rude,Whaaaat why,others
6,6,So whatsup,Nothing much. Sitting sipping and watching TV....,What are you watching on tv?,others
7,7,Ok,ok im back!!,"So, how are u",others
8,8,Really?,really really really really really,Y saying so many times...i can hear you,others
9,9,Bay,in the bay,😘 love you,others


In [3]:
df_test = functions.parse_file(r"testwithoutlabels.txt", "EmoContext")

In [4]:
text_data = []
for idx,row in df.iterrows():
    text_data.append(" {} fullstop {} fullstop {} fullstop".format(row['turn1'], row['turn2'], row['turn3']))

In [5]:
test_text_data = []
for idx,row in df_test.iterrows():
    test_text_data.append(" {} fullstop {} fullstop {} fullstop".format(row['turn1'], row['turn2'], row['turn3']))

In [6]:
emoticons_map = {
    '😘': ' emoticon',
    '😍': ' happyemoticon',
    '😁': ' happyemoticon',
    '😭': ' sademoticon',
    '😑': ' sademoticon',
    '😻': ' happyemoticon',
    '😂': ' happyemoticon',
    '👍': ' emoticon',
    '😀': ' happyemoticon',
    ':D': ' happyemoticon',
    '🙂':  ' happyemoticon',
    '<3': ' happyemoticon',
    '😓' : ' sademoticon',
    '😒' : ' angryemoticon',
    '😈' : ' emoticon',
    '👿' : ' angryemoticon',
    '🖑' : ' happyemoticon',
    '😾' : ' emoticon',
    '😠' : ' angryemoticon',
    '👻' : ' emoticon',
    ':(' : ' sademoticon',
    ':)' : ' happyemoticon',
    'xD' : ' happyemoticon',
    '💔' : ' sademoticon',
    '😥' : ' emoticon',
    '😞' : ' sademoticon',
    '😤' : ' angryemoticon',
    '😃' : ' happyemoticon',
    '😦' : ' sademoticon',
    ':3' : ' emoticon',
    '😼' : ' emoticon',
    '😏' : ' happyemoticon',
    '😱' : ' sademoticon',
    '😬' : ' sademoticon',
    '🙁' : ' sademoticon',
    '</3' : ' sademoticon',
    '😺' : ' happyemoticon',
    '😣' : ' angryemoticon',
    '😢' : ' sademoticon',
    '😆' : ' happyemoticon',
    '😄' : ' happyemoticon',
    '😅' : ' happyemoticon',
    ':-)' : ' happyemoticon',
    '😊' : ' happyemoticon',
    '😕' : ' sademoticon',
    '😽' : ' happyemoticon',
    '🙀' : ' angryemoticon',
    '🤣' : ' happyemoticon',
    '🤐' : ' emoticon',
    '😡' : ' sademoticon',
    '👌' : ' happyemoticon', 
    '😮' : ' emoticon',
    '❤️' : ' happyemoticon',
    '🙄' : ' happyemoticon',
    '😿' : ' sademoticon',
    '😉' : ' happyemoticon',
    '😋' : ' happyemoticon',
    '😐' : ' emoticon',
    '😹' : ' happyemoticon',
    '😴' : ' sademoticon',
    '💤' : ' emoticon',
    '😜' : ' happyemoticon',
    '😇' : ' happyemoticon',
    '😔' : ' sademoticon',
    '😩' : ' sademoticon',
    '❤' : ' happyemoticon',
    '😲' : ' emoticon',
    '😫' : ' sademoticon',
    '😳' : ' sademoticon',
    '😰' : ' sademoticon',
}

In [7]:
bad_words_df = pd.read_csv('bad-words.csv')
#bad_words_df['jigaboo']
bad_words = list(bad_words_df['jigaboo'])
bad_words.sort(key = lambda s: len(s))
bad_words = bad_words[::-1]

In [8]:
for i in range(0, len(text_data)):
    for k in emoticons_map.keys():
        if text_data[i].find(k) >= 0:
            text_data[i] = text_data[i].replace(k, emoticons_map[k])
    for bw in bad_words:
        bw = " {} ".format(bw)
        if text_data[i].find(bw) >= 0:
            text_data[i] = text_data[i].replace(bw, ' badword ')
            

In [9]:
for i in range(0, len(test_text_data)):
    for k in emoticons_map.keys():
        if test_text_data[i].find(k) >= 0:
            test_text_data[i] = test_text_data[i].replace(k, emoticons_map[k])
    for bw in bad_words:
        bw = " {} ".format(bw)
        if test_text_data[i].find(bw) >= 0:
            test_text_data[i] = test_text_data[i].replace(bw, ' badword ')
            

In [10]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from autocorrect import spell
#from spellchecker import SpellChecker

porter = PorterStemmer()
stop_words = set(stopwords.words('english'))
reserved_words = ['badword','fullstop','happyemoticon','sademoticon','angryemoticon']

#spell = SpellChecker()


def filter_text(text):    
    text = text.lower()
    tokens = word_tokenize(text)
    words = [word for word in tokens if word.isalpha()]
    #words = [w for w in words if not w in stop_words]
    stemmed = [porter.stem(word) for word in tokens]
    corrected = [spell(word) if not word in reserved_words else word for word in stemmed ]
    corrected = [word.lower() for word in corrected]
    #corrected = [spell.correction(word) for word in stemmed]

    return corrected

In [11]:
tokenized_text_data = []
test_tokenized_text_data = []

i=0
for sentence in text_data:
    tokenized_text_data.append(filter_text(sentence))
    if i % 100 == 0:
        print(i)
    i+=1

i=0
for sentence in test_text_data:
    test_tokenized_text_data.append(filter_text(sentence))
    if i % 100 == 0:
        print(i)
    i+=1

unique_words = {}

for words in tokenized_text_data:
    for word in words:
        if word not in unique_words:
            unique_words[word] = 1
        else:
            unique_words[word] += 1
            
for words in test_tokenized_text_data:
    for word in words:
        if word not in unique_words:
            unique_words[word] = 1
        else:
            unique_words[word] += 1

sorted_unique_words = sorted(unique_words.items(), key=lambda kv: kv[1], reverse=True)
token_values = {}
i = 1
for word in sorted_unique_words:
    token_values[word[0]] = i
    i += 1

final_tokenized_text_data = []
for i in range(0,len(tokenized_text_data)):
    sentence = []
    for word in tokenized_text_data[i]:
        sentence.append(token_values[word])
    final_tokenized_text_data.append(sentence)

final_test_tokenized_text_data = []
for i in range(0,len(test_tokenized_text_data)):
    sentence = []
    for word in test_tokenized_text_data[i]:
        sentence.append(token_values[word])
    final_test_tokenized_text_data.append(sentence)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [12]:
len(unique_words)

11191

In [13]:
sorted_unique_words[:50]

[('fullstop', 107007),
 ('a', 57464),
 ('i', 27692),
 ('you', 24954),
 ('not', 12283),
 ('is', 9658),
 ('to', 9438),
 ('happyemoticon', 9316),
 ('me', 9299),
 ('what', 8811),
 ('do', 8375),
 ('are', 8370),
 ('u', 8027),
 ('it', 7548),
 ('am', 7345),
 ('the', 5897),
 ('of', 5068),
 ('who', 4920),
 ('my', 4854),
 ('that', 4853),
 ('so', 4496),
 ('sademoticon', 4161),
 ('badword', 4077),
 ('your', 3801),
 ('and', 3702),
 ('know', 3657),
 ('...', 3496),
 ('no', 3426),
 ('have', 3338),
 ('in', 3268),
 ('how', 3153),
 ('like', 3109),
 ('for', 3059),
 ('ye', 3028),
 ('good', 2659),
 ('with', 2608),
 ('ok', 2581),
 ('but', 2562),
 ('love', 2505),
 ('can', 2497),
 ('be', 2482),
 ('just', 2455),
 ('about', 2208),
 ('want', 2174),
 ('talk', 2169),
 ('go', 2106),
 ('now', 2076),
 ('tell', 1970),
 ('will', 1866),
 ('all', 1803)]

In [14]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [15]:
NR_WORDS = len(unique_words)+1

In [16]:
len(final_test_tokenized_text_data)

5509

In [17]:
X = pad_sequences(final_tokenized_text_data)

In [18]:
words = {"others": 0, "angry": 1, "sad":2, "happy": 3}

def one_hot_vector(word):
    y = [0,0,0,0]
    y[words[word]] = 1
    return y

Y = []

for idx,row in df.iterrows():
    Y.append(one_hot_vector(row['label']))

Y = np.array(Y)

#### model

In [19]:
embed_dim = 256
lstm_out = 128
batch_size = 128

model = Sequential()
model.add(Embedding(NR_WORDS, embed_dim,input_length = X.shape[1]))
model.add(LSTM(lstm_out))
model.add(Dropout(0.3))
model.add(Dense(4,activation='softmax'))
adam = optimizers.Adam(lr=0.01)
rmsprop = optimizers.RMSprop(lr=0.005, rho=0.9, epsilon=None, decay=0.0)
model.compile(loss = 'binary_crossentropy', optimizer=rmsprop, metrics = ['accuracy', f1])

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 164, 256)          2865152   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 516       
Total params: 3,062,788
Trainable params: 3,062,788
Non-trainable params: 0
_________________________________________________________________


In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0, random_state = 42)

In [ ]:
model.fit(X_train, Y_train, batch_size = 256, nb_epoch = 3, verbose = 1)

In [ ]:
K.set_value(model.optimizer.lr, 0.0001)

In [23]:
from sklearn.model_selection import KFold
# prepare cross validation
kfold = KFold(n_splits=3)
Y_train = np.array(Y_train)
# enumerate splits
for train, validation in kfold.split(X_train):
    history = model.fit(X_train[train], Y_train[train],
                    validation_data=(X_train[validation], Y_train[validation]),
                    epochs=1, verbose=1, batch_size=128)

Train on 20106 samples, validate on 10054 samples
Epoch 1/1
 1408/20106 [=>............................] - ETA: 2:06 - loss: 0.1274 - acc: 0.9510 - f1: 0.8924

KeyboardInterrupt: 

#### Evaluation

In [ ]:
ev_result = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.3f" % (ev_result[0]))
print("Validation Accuracy: %.3f" % (ev_result[1]))
print("F1 score: %.3f" % (ev_result[2]))

In [ ]:
model_json = model.to_json()
with open("simple_model.json", "w") as outfile:
    outfile.write(model_json)
model.save_weights("simple_model.h5")

#### For submission

In [ ]:
X_test = pad_sequences(final_test_tokenized_text_data, maxlen=X.shape[1])

In [ ]:
res = model.predict(X_test, batch_size=128, verbose=1)

In [ ]:
len(X_test)

In [ ]:
revers_words = {0:"others", 1:"angry", 2:"sad", 3:"happy"}

def softmax_convert(res):
    max_i = 0
    max_v = 0
    for i in range(0,4):
        if res[i] > max_v:
            max_v = res[i]
            max_i = i
    return revers_words[max_i]

In [ ]:
results = []
for r in res:
    results.append(softmax_convert(r))

In [ ]:
df_test['label'] = results

In [ ]:
df_test.head(30)

In [ ]:
df_test.to_csv("test.txt",index=False , sep="\t")